# 🎯 Construção da Pré-lista UNIVERSE_CANDIDATES

Este notebook executa o fluxo completo:
1. **Ingestão** de dados de mercado (Yahoo Finance)
2. **Pipeline** UNIVERSE_CANDIDATES (filtros e classificação)
3. **Resumo** da pré-lista gerada

---


## 1. Setup e Imports


In [ ]:
import sys
import logging
from pathlib import Path
from datetime import datetime
import json

# Configura logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Adiciona módulos ao path
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(project_root / 'modules'))

print(f"📁 Project root: {project_root}")
print(f"📅 Data/hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
# Imports dos módulos do PortfolioZero
from portfoliozero.core.data.market_data_ingestion import (
    fetch_and_store_universe_market_data,
    validate_raw_market_data,
    load_data_source_config,
)

from portfoliozero.core.data.universe_candidates_pipeline import (
    build_universe_candidates,
    load_universe_candidates,
    validate_universe_candidates,
    get_pipeline_metadata,
)

import polars as pl

print("✅ Imports carregados com sucesso!")


## 2. Configuração

Carrega a configuração para ver os tickers que serão baixados.


In [ ]:
# Carrega configuração
config = load_data_source_config()

print(f"📊 Provider: {config.get('provider', 'N/A')}")
print(f"📈 Tickers configurados: {len(config.get('universe', []))}")
print(f"📅 Período: {config.get('date_range', {}).get('start', 'N/A')} a {config.get('date_range', {}).get('end', 'N/A')}")
print(f"\n🏷️ Primeiros 10 tickers:")
for ticker in config.get('universe', [])[:10]:
    print(f"   - {ticker}")
print(f"   ... e mais {len(config.get('universe', [])) - 10} tickers")


## 3. Ingestão de Dados de Mercado 📥

Baixa dados OHLCV do Yahoo Finance para todos os tickers configurados.

⚠️ **Atenção:** Esta etapa pode demorar 5-10 minutos dependendo da quantidade de tickers e conexão.


In [ ]:
# Define se deve sobrescrever dados existentes
OVERWRITE_EXISTING = False  # Mude para True se quiser re-baixar tudo

print("="*60)
print("INGESTÃO DE DADOS DE MERCADO")
print("="*60)
print(f"\n⚙️ Sobrescrever existentes: {OVERWRITE_EXISTING}")
print(f"🚀 Iniciando ingestão...\n")

# Executa ingestão
files = fetch_and_store_universe_market_data(overwrite=OVERWRITE_EXISTING)

print(f"\n✅ Ingestão concluída!")
print(f"📁 Arquivos gerados/atualizados: {len(files)}")


In [ ]:
# Valida dados brutos
validation = validate_raw_market_data()

print("\n📋 Validação dos dados brutos:")
print(f"   Válido: {'✅' if validation['valid'] else '❌'}")
print(f"   Arquivos: {validation.get('files', 0)}")
print(f"   Registros totais: {validation.get('total_records', 0):,}")
print(f"   Tickers únicos: {len(validation.get('tickers', []))}")

if validation.get('errors'):
    print(f"\n⚠️ Erros encontrados:")
    for err in validation['errors'][:5]:
        print(f"   - {err}")


## 4. Execução do Pipeline UNIVERSE_CANDIDATES 🔧

Aplica os filtros de seleção e gera a pré-lista de 60-80 candidatos.


In [ ]:
print("="*60)
print("PIPELINE UNIVERSE_CANDIDATES")
print("="*60)

# Executa pipeline
output_path = build_universe_candidates(force_refresh=True)

print(f"\n✅ Pipeline concluído!")
print(f"📁 Arquivo gerado: {output_path}")


## 5. Carrega e Analisa Resultado 📊


In [ ]:
# Carrega resultado
df = load_universe_candidates(output_path)

# Valida
result = validate_universe_candidates(df)

print(f"📊 Candidatos carregados: {len(df)}")
print(f"✅ Validação: {'OK' if result.is_valid else 'FALHOU'}")
if result.warnings:
    print(f"⚠️ Warnings: {len(result.warnings)}")
    for w in result.warnings:
        print(f"   - {w}")


In [ ]:
# Mostra primeiros registros
print("\n📋 Primeiros 15 candidatos (ordenados por volume):")
df.head(15)


## 6. Resumo da Pré-lista 📈


In [ ]:
# Constantes
TARGET_MIN = 60
TARGET_MAX = 80

total = len(df)
in_range = TARGET_MIN <= total <= TARGET_MAX

print("="*60)
print("RESUMO DA PRÉ-LISTA UNIVERSE_CANDIDATES")
print("="*60)

print(f"\n📊 Total de candidatos: {total}")
print(f"🎯 Intervalo alvo: {TARGET_MIN} - {TARGET_MAX}")

if in_range:
    print(f"✅ Dentro do intervalo: SIM")
else:
    print(f"❌ Dentro do intervalo: NÃO")
    if total < TARGET_MIN:
        print(f"   ⚠️ Faltam {TARGET_MIN - total} candidatos para atingir o mínimo")
    else:
        print(f"   ⚠️ Excedem {total - TARGET_MAX} candidatos acima do máximo")


In [ ]:
# Distribuição por setor
if 'setor' in df.columns and len(df) > 0:
    print("\n📊 Distribuição por SETOR:")
    sector_counts = df.group_by('setor').len().sort('len', descending=True)
    for row in sector_counts.iter_rows():
        setor, count = row
        pct = (count / total * 100) if total > 0 else 0
        print(f"   {setor}: {count} ({pct:.1f}%)")


In [ ]:
# Distribuição por volatilidade
if 'volatility_class' in df.columns and len(df) > 0:
    print("\n📊 Distribuição por VOLATILIDADE:")
    for vol_class in ['BAIXA', 'MEDIA', 'ALTA']:
        count = df.filter(pl.col('volatility_class') == vol_class).height
        pct = (count / total * 100) if total > 0 else 0
        print(f"   {vol_class}: {count} ({pct:.1f}%)")


In [ ]:
# Distribuição por liquidez
if 'liquidity_class' in df.columns and len(df) > 0:
    print("\n📊 Distribuição por LIQUIDEZ:")
    for liq_class in ['BAIXA', 'MEDIA', 'ALTA']:
        count = df.filter(pl.col('liquidity_class') == liq_class).height
        pct = (count / total * 100) if total > 0 else 0
        print(f"   {liq_class}: {count} ({pct:.1f}%)")


## 7. Metadados do Pipeline


In [ ]:
# Carrega metadados
metadata = get_pipeline_metadata(output_path)

if metadata:
    print("\n📋 Metadados do Pipeline:")
    print(f"   Execução: {metadata.execution_date}")
    print(f"   Registros entrada: {metadata.input_record_count:,}")
    print(f"   Registros saída: {metadata.output_record_count:,}")
    print(f"   Filtros aplicados: {len(metadata.filters_applied)}")
    
    print("\n   📌 Filtros:")
    for f in metadata.filters_applied:
        print(f"      - {f}")
    
    if metadata.warnings:
        print(f"\n   ⚠️ Warnings:")
        for w in metadata.warnings:
            print(f"      - {w}")


## 8. Visualização dos Candidatos


In [ ]:
# Exibe tabela completa
print(f"\n📋 LISTA COMPLETA DE CANDIDATOS ({len(df)} ativos):")
df


---

## 🎯 Próximos Passos

### Se dentro do intervalo (60-80):
1. ✅ Revisar distribuição setorial
2. ✅ Revisar perfil de risco (volatilidade)
3. ➡️ Prosseguir para **TASK_011** (seleção dos 30 supervisionados)

### Se fora do intervalo:
1. Ajustar tickers em `config/experiments/universe_data_sources_v1.yaml`
2. Ajustar filtros em `config/experiments/universe_selection_rules_v1.yaml`
3. Re-executar este notebook

---

### Arquivos de Configuração:
- **Tickers:** `config/experiments/universe_data_sources_v1.yaml`
- **Filtros:** `config/experiments/universe_selection_rules_v1.yaml`

### Arquivos Gerados:
- **Pré-lista:** `data/universe/UNIVERSE_CANDIDATES.parquet`
- **CSV:** `data/universe/UNIVERSE_CANDIDATES.csv`
